In [1]:
# https://github.com/aladdinpersson/Machine-Learning-Collection/blob/master/ML/TensorFlow/Basics/tutorial5-regularization.py

import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import cifar10

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"
os.environ["MKL_DEBUG_CPU_TYPE"] = "5"
os.environ["KMP_BLOCKTIME"] = "1"
os.environ["KMP_AFFINITY"] = "granularity=fine,compact,1,0"
os.environ["OMP_NUM_THREADS"] = "<num_threads>"

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
      for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            physical_devices = tf.config.list_physical_devices("GPU")
            tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
      print("No GPU device found")

No GPU device found


In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

In [4]:
# REMEMBER, NO COMMAS AFTER EACH FUNCTION CALL
def my_model():
    inputs = keras.Input(shape=(32, 32, 3))
    x = layers.Conv2D(
        32, 3,
        # don't shrink the layer
        padding="same",
        # regularizers reduce overfitting
        kernel_regularizer=regularizers.l2(0.01),
    )(inputs)
    # batch norm also has a regularizing effect.
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(
        64, 3,
        padding="same",
        kernel_regularizer=regularizers.l2(0.01)
    )(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(
        128, 3,
        padding="same",
        kernel_regularizer=regularizers.l2(0.01),
    )(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(
        64,
        activation="relu",
        kernel_regularizer=regularizers.l2(0.01)
    )(x)
    # dropout disables certain nodes at random during training, helps prevent overfitting.
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(10)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [5]:
model = my_model()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"]
)

In [6]:
# don't do 150 epochs you blithering idiot
# model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
# model.evaluate(x_test, y_test, batch_size=64, verbose=2)

model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=1)
model.evaluate(x_test, y_test, batch_size=64, verbose=1)

Epoch 1/10
782/782 [==============================] - 60s 76ms/step - loss: 2.9914 - accuracy: 0.3485
Epoch 2/10
782/782 [==============================] - 54s 68ms/step - loss: 1.8815 - accuracy: 0.4754
Epoch 3/10
782/782 [==============================] - 54s 69ms/step - loss: 1.6147 - accuracy: 0.5166
Epoch 4/10
782/782 [==============================] - 53s 67ms/step - loss: 1.4972 - accuracy: 0.5461
Epoch 5/10
782/782 [==============================] - 53s 67ms/step - loss: 1.4356 - accuracy: 0.5653
Epoch 6/10
782/782 [==============================] - 52s 67ms/step - loss: 1.3959 - accuracy: 0.5740
Epoch 7/10
782/782 [==============================] - 65s 83ms/step - loss: 1.3648 - accuracy: 0.5840
Epoch 8/10
782/782 [==============================] - 74s 95ms/step - loss: 1.3376 - accuracy: 0.5954
Epoch 9/10
782/782 [==============================] - 62s 79ms/step - loss: 1.3187 - accuracy: 0.6020
Epoch 10/10
157/157 [==============================] - 4s 21ms/step - loss: 1.3691

[1.3691333532333374, 0.6101999878883362]